In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from torch.optim import Adam
from tqdm import tqdm

from sklearn.model_selection import train_test_split
#GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
import random

SEED  = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
from sklearn.metrics import *

In [ ]:
from transformers import ElectraModel
from transformers import ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")


example_text = 'I will watch Memento tonight'
Electra_input = tokenizer(example_text,padding='max_length', max_length = 256, 
                       truncation=True, return_tensors="pt")

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

### 데이터 준비하기

In [ ]:
# 데이터를 각각 목적에 맞게 나누기
train_data = pd.read_excel('Dataset/Training.xlsx')
original_valid_data = pd.read_excel('Dataset/Validation.xlsx')

valid_data, test_data = train_test_split(original_valid_data,test_size=0.5, random_state=42, shuffle =True)

In [ ]:
labels = {
    "기쁨":0,
    "기쁨 ":0,
    "불안":1,
    "불안 ":1,
    "분노":2,
    "당황":3,
    "슬픔":4,
    "상처":5
}

In [ ]:
class ElectraDataset(Dataset):
    def __init__(self,df):
        self.labels = [labels[label] for label in df['감정_대분류']]
        self.texts = [tokenizer(str(sentence), padding = "max_length", max_length=128, truncation=True, return_tensors='pt') for sentence in df['사람문장1']]
        # print(self.texts)
    def classes(self):
        return self.labels
    
    def __len__(self):
        return len(self.labels)
    
    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])
    
    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]
    
    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [ ]:
class ElectraClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(ElectraClassifier, self).__init__()
        self.Electra  = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
        self.dropout = nn.Dropout(dropout)
        self.firstlinear = nn.Linear(768,1536)
        self.middlelinear = nn.Linear(1536,768)
        self.finallinear = nn.Linear(768, 6)
        self.relu  = nn.ReLU()
        self.softmax = nn.Softmax(dim=1) 

    def forward(self, input_id, mask):
        pooled_output = self.Electra(input_ids=input_id, attention_mask=mask,return_dict=False)
        
        x = self.dropout(pooled_output[0][:,0,:])
        x = self.firstlinear(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.middlelinear(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.finallinear(x)
        x = self.softmax(x)
        return x

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):
    global best_loss, patience_limit, patience_check

    train, val = ElectraDataset(train_data), ElectraDataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=32)
    
    use_cuda = torch.cuda.is_available()

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)    
    
    if use_cuda:
            criterion = criterion.cuda()
            
    for epoch_num in range(epochs):
            model.train()
            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                
                output= model(input_id, mask)

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
                
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
                
                
            total_acc_val = 0
            total_loss_val = 0
                
            model.eval()
            for val_input, val_label in val_dataloader:
                val_label = val_label.to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)
                mask = val_input['attention_mask'].to(device)
                output = model(input_id, mask)

                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()
                    
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
                

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [ ]:
EPOCHS = 5
model = ElectraClassifier()
model = model.to(device)
LR = 1e-5

train(model, train_data, valid_data, LR, EPOCHS)

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1278/1278 [12:56<00:00,  1.65it/s]


Epochs: 1 | Train Loss:  0.052                 | Train Accuracy:  0.358                 | Val Loss:  0.048                 | Val Accuracy:  0.494


100%|██████████| 1278/1278 [12:54<00:00,  1.65it/s]


Epochs: 2 | Train Loss:  0.046                 | Train Accuracy:  0.572                 | Val Loss:  0.045                 | Val Accuracy:  0.613


...

Epochs: 3 | Train Loss:  0.044                 | Train Accuracy:  0.645                 | Val Loss:  0.045                 | Val Accuracy:  0.624


100%|██████████| 1278/1278 [12:54<00:00,  1.65it/s]


Epochs: 4 | Train Loss:  0.043                 | Train Accuracy:  0.661                 | Val Loss:  0.044                 | Val Accuracy:  0.635


100%|██████████| 1278/1278 [12:54<00:00,  1.65it/s]


Epochs: 5 | Train Loss:  0.043                 | Train Accuracy:  0.676                 | Val Loss:  0.044                 | Val Accuracy:  0.640


In [ ]:
def evaluate(model, test_data):
    
    test = ElectraDataset(test_data)
    
    test_dataloader = torch.utils.data.DataLoader(test,batch_size=2)
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    total_acc_test =0
    
    total_output = []
    total_label = []
    for test_input, test_label in test_dataloader:            
        test_label = test_label.to(device)
        mask = test_input['attention_mask'].to(device)
        input_id = test_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        acc = (output.argmax(dim=1) == test_label).sum().item()
        total_acc_test += acc
        
        output_cpu = output.cpu().argmax(dim=1).tolist()
        test_label_cpu = test_label.cpu().tolist()
        total_output = total_output + output_cpu
        total_label = total_label + test_label_cpu
    
    total_output = torch.tensor(total_output)
    total_label = torch.tensor(total_label)
    
    print(classification_report(total_label,total_output))
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [ ]:
import time

global total_acc_test

start_time = time.time()
evaluate(model,test_data)
print("---{}s seconds---".format(time.time()-start_time))

              precision    recall  f1-score   support

           0       0.84      0.91      0.87       423
           1       0.64      0.71      0.67       456
           2       0.67      0.59      0.63       449
           3       0.70      0.47      0.56       405
           4       0.56      0.59      0.57       429
           5       0.49      0.57      0.53       403

    accuracy                           0.64      2565
   macro avg       0.65      0.64      0.64      2565
weighted avg       0.65      0.64      0.64      2565

Test Accuracy:  0.644
---22.226925134658813s seconds---
